Задание к лабораторной работе:
1. Выберите набор данных для проведения регрессионного анализа -  
   https://www.kaggle.com/slehkyi/extended-football-stats-for-european-leagues-xg
2. Проведите регрессионный анализ данных из выбранного набора.


Построим графики взаимного распределения совместных величин.

In [ ]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=2) # количество знаков после запятой
# встраивание рисунков
%matplotlib inline
df = pd.read_csv('../input/extended-football-stats-for-european-leagues-xg/understat.com.csv', encoding='utf-8', index_col=False, parse_dates=[0])
df = df[['wins','draws','loses', 'scored']]
df.head(5)

В датасете использовались такие данные как: position (позиция), team (команда), amount of matches played (количество сыгранных матчей), wins (победы), draws (ничьи), loses (поражения), goals scored (забитые голы), goals missed (пропущенные голы), points (очки).

In [ ]:
print(df.columns)

Построим графики взаимного распределения совместных величин. Среднее значение по этому параметру - 

In [ ]:
g=sns.pairplot(df[['wins','draws','loses', 'scored']])

График распределения величин. На пересечении - график совместных распределений. На диагонали - гистограммы.

In [ ]:
# Рисование регрессии от одной переменной к целевой.
plt.figure(figsize=(20,5))
for i, col in enumerate(['wins','draws','loses']):
    plt.subplot(1,3,i+1)
    x=df[col]
    y=df.scored
    plt.plot(x,y,'o') # отображение в виде кружков
    plt.plot(np.unique(x), np.poly1d(np.polyfit(x,y,1))(np.unique(x)),color='r') # нахождение регрессии с помощью многочлена (polyfit)
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('scored')

Разбиваем выборку с помощью train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
y=df['scored']
x=df.drop('scored',axis=1)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=0) # test_size - размер тестовой выборки;

Обучаем SGDRegressor - линейную регрессию.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn import  linear_model, metrics
regr=linear_model.SGDRegressor(random_state=42) # создание регрессора
regr.fit(x_train,y_train) # передаем обучающую выборку в модель (в переменную regr), т.е. обучаем модель
y_pred=regr.predict(x_test) # предсказание на основе x_test

print(metrics.mean_absolute_error(y_test,y_pred)) # абсолютная средняя ошибка

Абсолютная средняя ошибка примерно равна 16.9 

Сопоставляем данные:

In [ ]:
print(y_test[0:10].tolist()) # 10 изначальных значений
print(list(map(lambda x:"{:2e}".format(x),y_pred[0:10]))) # возвращение 10 предсказанных значений

In [ ]:
# Нормируем величины, т.е. приводим к одинаковой размерности, чтобы размерность величин меньше влияла на метод градиентного спуска.
from sklearn.preprocessing import StandardScaler # preprocessing - предподготовка данных
scaler=StandardScaler()
scaler.fit(x_train,y_train) # обучение на обучающих выборках
scaled_train_data=scaler.transform(x_train) # трансформируем x_train
scaled_test_data=scaler.transform(x_test) # трансформируем x_test

In [ ]:
regr.fit(scaled_train_data,y_train) # обучение линейной регрессии
y_scaled_pred=regr.predict(scaled_test_data) # новое предсказание с нормированными данными
print(metrics.mean_absolute_error(y_test,y_scaled_pred))

In [ ]:
# Отличие 5 реальных данных от 5 предсказанных.
print(y_test.values[:5])
print(y_scaled_pred[:5])

In [ ]:
# Построение графика.
# Чем лучше регрессия, тем более диагональным будет график.
import matplotlib.pyplot as plt
y_pred=regr.predict(x_test)
plt.scatter(y_test,y_scaled_pred,color="m")
plt.xlabel("Scores: $Y_i$")
plt.ylabel("Predicted scores: $\hat{Y}_i$")
plt.title("Scores vs Predicted scores: $Y_i$ vs $\hat{Y}_i$")

In [ ]:
# Нормализация и обучение линейной регрессии в кратком виде.
from sklearn.pipeline import Pipeline
pipe=Pipeline(steps=[('scaling',scaler),('regression',regr)])
pipe.fit(x_train,y_train)
print(metrics.mean_absolute_error(y_test,pipe.predict(x_test))) # метрика абсолютной ошибки

Метапараметры модуля Pipeline

In [ ]:
pipe.get_params().keys()

In [ ]:
# Подбор параметров при которых обучение происходит лучше всего.
from sklearn.model_selection import GridSearchCV

parameters_grid={
    'regression__loss':['huber','squared_loss',],
    'regression__n_iter_no_change':[ 5,10,20],
    'regression__penalty':['l1','l2','none'],
    'regression__alpha':[0.0001,0.001,0.1],
}
# на основе значений формируется многомерная сетка
grid_cv=GridSearchCV(pipe,parameters_grid,scoring='neg_mean_absolute_error',cv=4)
# выборка на этих данных какие значения будут лучше
grid_cv.fit(x_train, y_train)

In [ ]:
print(-1*grid_cv.best_score_) # лучшая оценка grid_cv
print(grid_cv.best_params_) # лучшие параметры grid_cv

In [ ]:
# Построение графика с улучшенными значениями.
plt.scatter(y_test,pipe.predict(x_test),color="b",alpha=0.6)
plt.xlabel("Scores: $Y_i$")
plt.ylabel("Predicted scores: $\hat{Y}_i$")
plt.title("Scores vs Predicted scores: $Y_i$ vs $\hat{Y}_i$")

Какого-то качественного скачка в сторону улучшения модели не произошло.

In [ ]:
# Сравниваем графики.
plt.scatter(y_test,pipe.predict(x_test),color="b",alpha=0.6)
plt.scatter(y_test,grid_cv.best_estimator_.predict(x_test),color="r",alpha=0.6)
plt.xlabel("Scores: $Y_i$")
plt.ylabel("Predicted scores: $\hat{Y}_i$")
plt.title("Scores vs Predicted scores: $Y_i$ vs $\hat{Y}_i$")
plt.show()

In [ ]:
# Построение графика методом Пирсона.
x_corr=x_train.corr(method="pearson") # коэффициента корреляции x_corr
mask=np.zeros_like(x_corr) # маска
mask[np.triu_indices_from(mask)]=True
fig,ax=plt.subplots(figsize=(10,8))
fig=sns.heatmap(x_corr,cmap="RdYlGn_r",mask=mask) # тепловая карта корреляции

plt.show()

Сильная отрицательная корреляция у параметров loses и wins. 

In [ ]:
# Удаление параметра loses
x_train_dropped=x_train.drop('loses',axis=1)
x_test_dropped=x_test.drop('loses',axis=1)
grid_cv_drpd=grid_cv
grid_cv_drpd.fit(x_train_dropped,y_train)
print(-1*grid_cv_drpd.best_score_) # вывод новой ошибки и нового лучшего параметра
print(grid_cv_drpd.best_params_)

In [ ]:
# Графики сравнения.
plt.scatter(y_test,pipe.predict(x_test),color="b",alpha=0.6)
plt.scatter(y_test,grid_cv.best_estimator_.predict(x_test_dropped),color="r",alpha=0.6)
plt.xlabel("Scores: $Y_i$")
plt.ylabel("Predicted scores: $\hat{Y}_i$")
plt.title("Scores vs Predicted scores: $Y_i$ vs $\hat{Y}_i$")
plt.show()

Наблюдаем небольшие изменения.

In [ ]:
# Возьмем другую модель для обучения - модель регрессии при помощи случайного леса.
from sklearn.ensemble import RandomForestRegressor

rfr=RandomForestRegressor(random_state=42)
rfr.fit(x_train,y_train)
rf_predict=rfr.predict(x_test)
print(metrics.mean_absolute_error(y_test,rf_predict))

grid_cv=GridSearchCV(pipe,parameters_grid,scoring='neg_mean_absolute_error',cv=4)
grid_cv.fit(x_train,y_train)
plt.scatter(y_test,rf_predict,color="r",alpha=0.6)